In [93]:
import numpy as np
from numpy.linalg import eigvals
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit.opflow import PrimitiveOp, PauliTrotterEvolution
from qiskit.circuit import Parameter
from qiskit import transpile, QuantumCircuit

t = Parameter('t')

def trotter_step(ops, coe):
    """Returns a single trotter step consisting only of CNOT and single qubit gates.
        ops: list of Pauli strings
        coe: list of corresponding coefficients for each Pauli string
    """
    ham = PrimitiveOp(SparsePauliOp(ops, coe)) # Hamiltonian
    U = (t*ham).exp_i() # Unitary parameterized by t
    Ut = PauliTrotterEvolution().convert(U) # single Trotter step
    qc = transpile(Ut.to_circuit(), basis_gates=['u','cx']) # turn into circuit with only allowed gates
    return qc

def create_circuit(ops, coe, steps):
    """Returns a trotterized circuit
        ops: list of Pauli strings
        coe: list of corresponding coefficients for each Pauli string
        steps: number of trotter steps
    """
    ts = trotter_step(ops, coe).bind_parameters({t: 1/steps}) # trotter step
    nq = ts.width() # number of qubits
    qc = QuantumCircuit(nq)
    for i in range(steps):
        qc.append(ts, range(nq))
    return qc

def create_unitary(ops, coe):
    """Returns a unitary matrix
        ops: list of Pauli strings
        coe: list of corresponding coefficients for each Pauli string
    """
    return PrimitiveOp(SparsePauliOp(ops, coe)).exp_i().to_matrix()

def circuit_error(circuit, unitary):
    """Return error defined as abs(largest eigenvector) of Uc-U where
       Uc is the unitary representing the circuit and U=exp(-iH) the
       exact unitary matrix attempting to be recreated
        circuit: quantum circuit object (must be unitary, parameters must be bound)
        unitary: matrix
    """
    Uc = Operator(circuit).data
    return np.max(np.abs(eigvals(Uc - unitary)))
    

ops = ['IIX','XXI','ZZZ']
coe = [.5,.1,.3]
circ = create_circuit(ops, coe, 20)
#print(circ)
u = create_unitary(ops, coe)
print(circuit_error(circ,u))

0.007082514526657972


In [39]:
circ.depth()

10